In [0]:
# If notebook is in the repo root, and dashboard is in dashboards/
import os, json, base64, requests
from datetime import datetime

# Get user and context info
user = spark.sql("SELECT current_user()").collect()[0][0]
ctx = dbutils.notebook.entry_point.getDbutils().notebook().getContext()
api_url = ctx.apiUrl().get()
token = ctx.apiToken().get()

In [ ]:
# Get current notebook path and compute dashboards folder
notebook_path = ctx.notebookPath().get()
base_path = "/".join(notebook_path.split("/")[:-1])
dashboard_folder = os.path.join(os.getcwd(), "dashboards")
workspace_base_path = f"{base_path}/dashboards"

# Generate timestamp for unique dashboard naming
timestamp = datetime.now().strftime("%Y%m%d_%H%M")
print(f"🕐 Installation timestamp: {timestamp}")

print(f"📂 Dashboard folder path: {dashboard_folder}")
print(f"📁 Found {len(os.listdir(dashboard_folder))} file(s) in {dashboard_folder}")

# Log install info - Comment out the next line to disable telemetry
ENABLE_TELEMETRY = True  # Set to False or comment out to disable usage analytics

if ENABLE_TELEMETRY:
    try:
        # Extract workspace ID from the API URL or notebook path
        workspace_id = api_url.split("/")[2].split(".")[0]  # Gets workspace from URL like https://dbc-abc123.cloud.databricks.com
        
        # Mask the email - only mask first 5 characters
        if '@' in user:
            username, domain = user.split('@')
            if len(username) > 5:
                masked_user = '*' * 5 + username[5:] + '@' + domain
            else:
                masked_user = '*' * len(username) + '@' + domain
        else:
            masked_user = user  # If no @ symbol, use as is
        
        log_response = requests.post(
            'http://87.121.93.91:8080/api/log',
        headers={
            'x-api-key': 'chaplin',
            'Content-Type': 'application/json'
        },
            json={
                'workspace_id': workspace_id,
                'user': masked_user  # Send masked email instead of full email
            },
            timeout=10
        )
        
        if log_response.status_code == 201:
            print(f"📊 Activity logged successfully for workspace: {workspace_id}, user: {masked_user}")
        else:
            print(f"⚠️ Failed to log activity: {log_response.status_code}")
            
    except Exception as e:
        print(f"⚠️ Logging error: {str(e)}")

for fname in os.listdir(dashboard_folder):
    if fname.endswith(".lvdash.json"):
        base_dashboard_name = fname.replace(".lvdash.json", "")
        # Add timestamp to make dashboard name unique
        dashboard_name = f"{base_dashboard_name}_{timestamp}"
        
        print(f"\n📊 Processing: {base_dashboard_name}")
        print(f"   ➡️  Will create as: {dashboard_name}")
        
        with open(os.path.join(dashboard_folder, fname), "r", encoding="utf-8") as f:
            dashboard_def = json.load(f)

        # First, try to create the dashboard
        response = requests.post(
            url=f"{api_url}/api/2.0/lakeview/dashboards",
            headers={
                "Authorization": f"Bearer {token}",
                "Content-Type": "application/json"
            },
            json={
                "display_name": dashboard_name,
                "parent_path": workspace_base_path,
                "serialized_dashboard": json.dumps(dashboard_def),
                "warehouse_id": None  # Will use default warehouse
            }
        )

        if response.status_code == 200 or response.status_code == 201:
            result = response.json()
            dashboard_id = result.get("dashboard_id", "")
            dashboard_path = result.get("path", "")
            print(f"✅ Created Lakeview Dashboard: {dashboard_name}")
            print(f"   📊 Dashboard ID: {dashboard_id}")
            print(f"   🔗 Path: {dashboard_path}")
            print(f"   🌐 URL: {api_url}/sql/dashboardsv3/{dashboard_id}")
        elif response.status_code == 400:
            # Check if it's a RESOURCE_ALREADY_EXISTS error
            try:
                error_json = response.json()
                if error_json.get("error_code") == "RESOURCE_ALREADY_EXISTS":
                    print(f"⚠️  Dashboard '{dashboard_name}' already exists. Finding and updating...")
                    
                    # List all dashboards to find the existing one
                    list_response = requests.get(
                        url=f"{api_url}/api/2.0/lakeview/dashboards",
                        headers={"Authorization": f"Bearer {token}"}
                    )
                    
                    if list_response.status_code == 200:
                        dashboards = list_response.json().get("dashboards", [])
                        existing_dashboard = None
                        
                        # Find the dashboard by name
                        for dash in dashboards:
                            if dash.get("display_name") == dashboard_name or dash.get("display_name") == f"{dashboard_name}.lvdash.json":
                                existing_dashboard = dash
                                break
                        
                        if existing_dashboard:
                            dashboard_id = existing_dashboard["dashboard_id"]
                            
                            # Update the existing dashboard using PATCH
                            update_response = requests.patch(
                                url=f"{api_url}/api/2.0/lakeview/dashboards/{dashboard_id}",
                                headers={
                                    "Authorization": f"Bearer {token}",
                                    "Content-Type": "application/json"
                                },
                                json={
                                    "serialized_dashboard": json.dumps(dashboard_def)
                                }
                            )
                            
                            if update_response.status_code == 200:
                                result = update_response.json()
                                print(f"✅ Updated Lakeview Dashboard: {dashboard_name}")
                                print(f"   📊 Dashboard ID: {dashboard_id}")
                                print(f"   🔗 Path: {existing_dashboard.get('path', '')}")
                                print(f"   🌐 URL: {api_url}/sql/dashboardsv3/{dashboard_id}")
                            else:
                                print(f"❌ Failed to update dashboard: {dashboard_name} — Status: {update_response.status_code}")
                                print(f"Error Response: {update_response.text}")
                        else:
                            print(f"❌ Could not find existing dashboard: {dashboard_name}")
                    else:
                        print(f"❌ Failed to list dashboards — Status: {list_response.status_code}")
                else:
                    print(f"❌ Failed to import: {dashboard_name} — Status: {response.status_code}")
                    print(f"Error Response: {response.text}")
                    print(f"Error Details: {json.dumps(error_json, indent=2)}")
            except:
                print(f"❌ Failed to import: {dashboard_name} — Status: {response.status_code}")
                print(f"Error Response: {response.text}")
        else:
            print(f"❌ Failed to import: {dashboard_name} — Status: {response.status_code}")
            print(f"Error Response: {response.text}")
            try:
                error_json = response.json()
                print(f"Error Details: {json.dumps(error_json, indent=2)}")
            except:
                pass